# Fine-mapping with SuSiE model

This notebook performs statistical fine-mapping using SuSiE model for individual level data. 

## Input

1. A list of regions to be analyzed.
2. A list of genotype files per region to be analyzed, in PLINK `bed` format. 
3. lists of phenotype files per region to be analyzed, in UCSC `bed.gz` with index in `bed.gz.tbi` formats.
4. vector of covariate files corresponding to the lists above.

Input 2 and 3 should be outputs from `genotype_per_region` and `phenotype_per_region` modules in previous preprocessing steps. 4 should be output of `covariate_preprocessing` pipeline that contains genotype PC, phenotypic hidden confounders and fixed covariates.

### Example genotype list

```
# region        dir
ENSG00000000457 /mnt/mfs/statgen/xqtl_workflow_testing/genotype_per_region/ENSG00000000457.bed
ENSG00000000460 /mnt/mfs/statgen/xqtl_workflow_testing/genotype_per_region/ENSG00000000460.bed
ENSG00000000938 /mnt/mfs/statgen/xqtl_workflow_testing/genotype_per_region/ENSG00000000938.bed
ENSG00000000971 /mnt/mfs/statgen/xqtl_workflow_testing/genotype_per_region/ENSG00000000971.bed
ENSG00000001036 /mnt/mfs/statgen/xqtl_workflow_testing/genotype_per_region/ENSG00000001036.bed
ENSG00000001084 /mnt/mfs/statgen/xqtl_workflow_testing/genotype_per_region/ENSG00000001084.bed
ENSG00000001167 /mnt/mfs/statgen/xqtl_workflow_testing/genotype_per_region/ENSG00000001167.bed
ENSG00000001460 /mnt/mfs/statgen/xqtl_workflow_testing/genotype_per_region/ENSG00000001460.bed
```

### Example phenotype list

```
# region dir
ENSG00000000457 /mnt/mfs/statgen/xqtl_workflow_testing/phenotype_per_region/ENSG00000000457.t1.bed.gz
ENSG00000000460 /mnt/mfs/statgen/xqtl_workflow_testing/phenotype_per_region/ENSG00000000460.t1.bed.gz
ENSG00000000938 /mnt/mfs/statgen/xqtl_workflow_testing/phenotype_per_region/ENSG00000000938.t1.bed.gz
ENSG00000000971 /mnt/mfs/statgen/xqtl_workflow_testing/phenotype_per_region/ENSG00000000971.t1.bed.gz
ENSG00000001036 /mnt/mfs/statgen/xqtl_workflow_testing/phenotype_per_region/ENSG00000001036.t1.bed.gz
ENSG00000001084 /mnt/mfs/statgen/xqtl_workflow_testing/phenotype_per_region/ENSG00000001084.t1.bed.gz
ENSG00000001167 /mnt/mfs/statgen/xqtl_workflow_testing/phenotype_per_region/ENSG00000001167.t1.bed.gz
ENSG00000001460 /mnt/mfs/statgen/xqtl_workflow_testing/phenotype_per_region/ENSG00000001460.t1.bed.gz
```

## Output

For each analysis region, the output is SuSiE model fitted and saved in RDS format.

## Minimal working example

```
# suggested output naming convention is cohort_modality, eg ROSMAP_snRNA_pseudobulk
sos run SuSiE.ipynb susie \
    --name cohort_modality \
    --genoFile /path/to/genotype_list \
    --phenoFile /path/to/phenotype_list_1 /path/to/phenotype_list_1 /path/to/phenotype_list_3 \
    --covFile phenotype_1_cov phenotype_2_cov phenotype_3_cov \
    --utils-R /path/to/misc/xqtl_utils.R \
    --container /path/to/stephenslab.sif
```

In [1]:
[global]
import glob
import pandas as pd
# A list of file paths for genotype data. 
parameter: genoFile = path
# One or multiple lists of file paths for phenotype data.
parameter: phenoFile = paths
# Covariate file path
parameter: covFile = path
# Optional: if a region list is provide the analysis will be focused on provided region. 
# Otherwise, all regions with both genotype and phenotype files will be analyzed
parameter: region_list = path()
parameter: cwd = path("output")
# It is required to input the name of the analysis
parameter: name = str
# path to utility script. In the future we will consolidate this into an R package.
parameter: utils_R = path("misc/xqtl_utils.R")
parameter: container = ""
parameter: entrypoint={('micromamba run -a "" -n' + ' ' + container.split('/')[-1][:-4]) if container.endswith('.sif') else f''}
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 2

utils_R = f"{utils_R:a}"

## Univariate SuSiE

In [ ]:
[get_analysis_regions: provides = "regional_data"]
# regional_data should be a dictionary with:
# 1. a list of tuples: {data: [(gene_1.genotype, gene_1.condition_1, cov_1, gene_1.condition_2, cov_2), (gene_2.genotype, gene_2.condition_1, cov_1, gene_2.condition_2, cov_2), ...]}
# 2. a list of region ID: {ID: [gene_1, gene_2, ...]}
# FIXME: Hao, please implement the details
regional_data = {"data": [(), ()], "ID": ["gene_1", "gene_2"]}

In [1]:
[susie_1]
parameter: max_L = 20
# remove a variant if it has more than imiss missing individual level data
parameter: imiss = 0.1
# MAF cutoff
parameter: maf = 0
# MAC cutoff, on top of MAF cutoff
# Here I set default to mac = 10 rather than using an MAF cutoff
# I don't set it to 5 because I'm not so sure of performance of SuSiE on somewhat infrequent variants
# MAC = 10 would not be too infrequenty for xQTL data where sample size is about ~1,000 at most (as of 2022)
paramter: mac = 10
depends: sos_variable("regional_data")
input: regional_data["data"], group_by = lambda x: ...
output: f'{cwd:a}/SuSiE/{name}.{regional_data[_index]}.susie_fitted.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint, input = utils_R
    fdat = load_regional_association_data(genotype = ${_input[0]:ar}, 
                                          phenotype = c(${_input[1::2]:ar,}), 
                                          covariate = c(${_input[2::2]:ar,}),
                                          maf_cutoff = ${maf},
                                          mac_cutoff = ${mac},
                                          imiss_cutoff = ${imiss})
    # Fine-mapping with SuSiE
    library("susieR")
    fitted = list()
    for (r in colnames(fdat$residual_Y)) {
        ## FIXME: Hao, after you finish xqtl_utils.R please check my logic of the next 3 lines of code
        non_missing = which(!is.na(fdat$residual_Y[,r]))
        y = fdat$residual_Y[non_missing,r,drop=F]
        X = fdat$residual_X[[r]][rownames(y),]
        print(paste("Dimension of X matrix:", r, nrow(X), ncol(X)))
        st = proc.time()
        fitted[[r]] <- susie(X,y,
                             L=${max_L},
                             max_iter=1000,
                             estimate_residual_variance=TRUE,
                             estimate_prior_variance=TRUE,
                             refine=TRUE,
                             compute_univariate_zscore=TRUE, 
                             coverage=0.95)
        fitted[[r]]$time = proc.time() - st
        fitted[[r]]$cs_corr = get_cs_correlation(fitted[[r]], X=X)
        fitted[[r]]$cs_snps = names(fitted[[r]]$X_column_scale_factors[unlist(fitted[[r]]$sets$cs)])
        fitted[[r]]$variable_name = names(fitted[[r]]$pip)
        fitted[[r]]$coef = coef.susie(fitted[[r]])
    }
    saveRDS(fitted, ${_output:ar})

In [ ]:
[susie_2]
input: group_by = "all"
output: f'{cwd}/{name}.susie_output.txt'
python: expand= "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout', container = container, entrypoint = entrypoint
    import pandas as pd
    pd.DataFrame({"output" : [$[_input:ar,]]}).to_csv("$[_output]",index = False ,header = False, sep = "\t")